<p style="color:red">Important note: the code snippets provided below correspond to a system where (0th axis = x axis) and (1st axis = y axis).</p>
<p style="color:red"> If you adopt a different convention in your simulation, please modify the code snippets accordingly!</p>

### 1) Start your MPI communicator.

In [ ]:
comm = MPI.COMM_WORLD
size = comm.Get_size() # size is the total number of the cores.
rank = comm.Get_rank() # rank is the id of each core.

### 2) Divide your simulation domain into subdomains.

In [ ]:
# Divide the simulation cell into (Nsub_X x Nsub_Y) subdomains.
Nsub_Y = int(np.floor(np.sqrt(size))) 
Nsub_X = int(size//Nsub_y)

Note: the above domain decomposition only applies for a symmetric grid.

### 3) Define the grid size of each subdomain .
Note: Don't forget to add 2 buffer cells in each direction.

### 4) Start a cartesian communicator and get the coordinates of each core.

In [ ]:
cartcomm=comm.Create_cart(dims=[Nsub_X,Nsub_Y],periods=[False,False],reorder=False)
rcoords = cartcomm.Get_coords(rank)

Make sure that you understand how the coordinates of the cores map to the physical domain of your system.

### 5) Define source / destination for each core.

In [ ]:
# Send to right and receive from left. sR corresponds to source and dR corresponds to destination.
sR,dR = cartcomm.Shift(0,1)

#Send to up and receive from down.
sU,dU = cartcomm.Shift(1,1)

sd = [sR,dR,sL,dL,sU,dU,sD,dD] # At the end, gather up all the sources and destinations into a single list.

### 6) Define the boundaries of each subdomain.

In [ ]:
boundary = [False,False,False,False] # [Left,Right,Down,Up].

if rcoords[0] == 0: boundary[0] = True # if True then there is a boundary to the left of this subdomain.
if rcoords[1] == Nsub_Y-1: boundary[3] = True # if True then there is a boundary to the top of this subdomain.

### 7) Define a function to handle communication between the cores.

In [ ]:
def Communicate(f_ikl,cartcomm,sd):
    sR,dR,sL,dL,sU,dU,sD,dD = sd
    
    # Send to left and receive from right.
    Lrcb = f_ikl[:,-1,:].copy() # Lrcb is an array which will receive information from the right.
    
    cartcomm.Sendrecv(f_ikl[:,1,:].copy(), dL, recvbuf = Lrcb, source = sL)
    f_ikl[:,-1,:] = Lrcb # We copy Lrcb to its corresponding location in the distribution function (f).


### 8) Start iterating the solution.

Note: you have to Communicate information between the cores right before streaming!.


In [ ]:
# Apply bounce-back boundary condition on the left boundary

if boundary[0] == True : f_ikl = BounceBackL(f_ikl) # BounceBack is the function which handles the no-slip boundary conditions on the left